In [2]:
# Иерархические методы

In [3]:
# Самая простая иерархичесая структура - это древо решений.
# Древа решений применяются для решения задач классификации и прогнозирования.
# В наиболее простом виде древо решений это ответы на вопросы да и нет.
# (Посмотри Задачу классификации цетов ириса (Fisher, 1936) ((Есть такой датасет)))
# Деревья решений рисуют линии строго препендикулярно осям
# Преимущества деревьев решений:
#   Интуитивность и интерпретированность деревьев решений
#   Деревья решений дают возможность извлекать правила из базы данных на естественном языке
#   Алгоритм конструирования дерева решений не требует от пользователя выбора входных атрибутов (Независимых переменных)
#   Точность моделей
#   Разработаны ряд масштабируемых алгоритмов
#   Быстрый процесс обучения
#   Имеют возможность специальной обработки пропущенных значений
#   Могут работать и с числовыми и с категориальными типами данных
# Недостатки деревьев решений:
#   Некоторые алгоритмы например: Исключающая или
#   Плохая предсказательная (обобщающая) способность

In [4]:
import pandas as pd #импорт библиотек, которые потребуются.
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO
from IPython.display import Image

%matplotlib inline # Вообще нам сказали, что это для настройки графика по факту хз

plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (18,12)

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

UsageError: unrecognized arguments: # Вообще нам сказали, что это для настройки графика по факту хз


In [ ]:
df_churn = pd.read_csv('dir_for_csv_file/Модуль_2_Тема_2_Отток.csv') # загрузим данные из файла (данные по оттоку клиентов)

In [ ]:
df_churn.head().transpose() #Посмотрим на наши данные, видно, что они нуждаются в обработке.

In [ ]:
df_churn.info()

In [ ]:
def preproc(df_init: pd.DataFrame): # Объявляем функцию
    df_preproc = df_init.copy()
    df_preproc = df_preproc.drop(['State', 'Area Code', 'Phone', 'Account Length', 'Intl Charge',
                                  'Day Calls', 'Eve Calls', 'VMail Message'], axis=1) # удаляем малоинформативные столбцы

    df_preproc.loc[:,  ["Int'l Plan", 'VMail Plan']] = \
    df_preproc.loc[:,  ["Int'l Plan", 'VMail Plan']].replace({'no': 0, 'yes': 1}) # делаем замену значений в указанных столбцах

    df_preproc.loc[:,  'Churn?'] = df_preproc.loc[:,  'Churn?'].replace({'False.': 0, # аналогично делаем замену и в столбце 'Churn?'
                                                                         'True.': 1})
    return df_preproc

In [ ]:
df_preproc = df_churn.pipe(preproc) # обрабоатываем датафрейм при помощи функции, описанной выше.
df_preproc.info() # смотрим на наши данные - пропусков нет, и среди данных только числовые значения.

In [ ]:
X, y = df_preproc.iloc[:, :-1].values, df_preproc.iloc[:, -1].values # разделяем датафрейм на два множества:
# данные для обучения и ответы.

In [ ]:
X.shape
X

In [ ]:
y.shape

In [ ]:
type(X)

# Кросс-валидация по одному гиперпараметру

In [ ]:
from sklearn.model_selection import validation_curve
from sklearn.model_selection import StratifiedKFold

In [ ]:
from sklearn.tree import DecisionTreeClassifier #импортируем модель деревьев решений
model = DecisionTreeClassifier(random_state=123) # инициализируем модель

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=132) #разбиваем наше множество на 5 частей и перемешиваем "shuffle=True"

train_scores, valid_scores = validation_curve(model, X, y,       #задаем параметры для валидационной кривой.
                                              param_name='max_depth', param_range=range(1, 20),
                                              cv=cv, scoring='accuracy')

In [ ]:
train_scores.shape

In [ ]:
valid_scores.shape


In [ ]:
train_score_mean = train_scores.mean(axis=1)
train_score_std = train_scores.std(axis=1)
valid_scores_mean = valid_scores.mean(axis=1)
valid_scores_std = valid_scores.std(axis=1)

plt.fill_between(range(1,20), train_score_mean-train_score_std, train_score_mean+train_score_std, color='b',
                 interpolate=True, alpha=0.5,)
plt.fill_between(range(1,20), valid_scores_mean-valid_scores_std, valid_scores_mean+valid_scores_std, color='r',
                 interpolate=True, alpha=0.5)

plt.plot(range(1,20), train_score_mean, c='b', lw=2)
plt.plot(range(1,20), valid_scores_mean, c='r', lw=2)
print(max(valid_scores_mean))

plt.xlabel('max depth')
plt.ylabel('ROC AUC')

In [ ]:

from sklearn.model_selection import RandomizedSearchCV #импортирует случайный поиск

RND_SEED = 123

In [ ]:
param_grid = {                                 #зададим параметры по которым и будем осуществлять поиск
    'criterion': ['gini', 'entropy'],
    'max_depth': range(2, 8),
    'min_samples_leaf': range(5, 10),
    'class_weight': [None, 'balanced']}

cv = StratifiedKFold(n_splits=5, random_state=123, shuffle=True)

model = DecisionTreeClassifier(random_state=123)
random_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=100, n_jobs=-1,
                                   cv=cv, scoring='roc_auc', random_state=123)

random_search.fit(X, y)

In [ ]:
random_search.best_params_ #выведем наилучшие параметры

In [ ]:
random_search.best_score_ #лучшее значение метрики

In [ ]:
best_model = random_search.best_estimator_;
best_model #параметры наилучшей модели

In [ ]:
model = random_search.best_estimator_
imp = model.feature_importances_

In [ ]:
pd.Series(index=df_preproc.columns[:-1], data = imp).sort_values()